# Customer Lookup Analytic

In this notebook, we will go through the steps needed to build the `.air` file which configures using the cusotomer lookup remote analytic.

## Package Installation and Import

First, we import the `aisquared` package

In [2]:
# import
import aisquared

## Configure the steps required

The workflow of using the `aisquared` package involves the creation of steps, in the following order:

- harvesting
- preprocessing
- analytic/model
- postprocessing
- rendering
- feedback_steps

In [3]:
# Because this specific analytic requires 
harvester = aisquared.config.harvesting.TextHarvester(
    how = 'regex',   # how to harvest the text
    regex = 'C\d{5}' # regex to harvest using
)

# There are also no preprocessing steps for this analytic
preprocesser = None

# For the analytic step, we need the DeployedAnalytic class
analytic = aisquared.config.analytic.DeployedAnalytic(
    url = "https://ifswlpc5q9.execute-api.us-east-2.amazonaws.com/test/returncustomeridresource?customer=",
    input_type = 'text',
    secret = '',
    header = None
)

# No posptprocessing is required
postprocesser = None

# The WordRendering class is used for rendering
renderer = aisquared.config.rendering.WordRendering(
    word_list = 'input',   # how to retrieve the words to use
    result_key = None,     # how to retrieve results
    content_key = 'nba',   # how to retrieve the contents
    badge_shape = 'star',  # badge shape to use
    badge_color = 'blue'   # badge color to use
)

# The SimpleFeedback class is used for rendering prediction correction feedback
model_feedback = aisquared.config.feedback.ModelFeedback()
model_feedback.add_question('Does this model help you in your workflow?', 'singleChoice', ['yes', 'no'])
feedback = aisquared.config.feedback.SimpleFeedback()

## Compile all Steps as a ModelConfiguration Object

After the steps have been initiated, the entire `ModelConfiguration` object can be constructed and the `.compile()` method run to create the `.air` file.

In [4]:
# Initialize the ModelConfiguration object
config = aisquared.config.ModelConfiguration(
    name = 'CustomerLookup',                     # the name of the model
    harvesting_steps = harvester,                # the harvester
    preprocessing_steps = preprocesser,          # the preprocesser
    analytic = analytic,                         # the analytic
    postprocessing_steps = postprocesser,        # the postprocessing steps
    rendering_steps = renderer,                  # the rendering steps
    feedback_steps = [model_feedback, feedback]  # prediction feedback
)

# Run the .compile() methods to create the .air file
config.compile()

## Upload the model to remote storage

After the model has been compiled and the `.air` file created, we can use the `AWSClient` object to upload the model to remote storage.

In [5]:
# Create the AWSClient object
client = aisquared.remote.AWSClient()
client.upload_model('CustomerLookup.air', overwrite = True)